# CDMA Despreading Code Mapping

The "Map" block in GNU Radio can easily spread a binary symbol into two selected codewords by using a map of [codeword0, codeword1]. To do the opposite operation, though, a map of all the possible values of len(codeword) chips onto 0 and 1 is needed. This map needs to make sense for any possible value, because there will be demodulation errors. To make sense, it should map each received value onto the value for the nearest codeword (by Hamming distance). For ties, it should do something fair, like passing through the least significant bit in which the codewords differ.

This script does just that, creating the map table to be used by a GNU Radio "Map" block to convert demodulated chips into symbols.

The symbols are treated as binary, and are mapped to 0 or 1, but since they are packed into bytes it doesn't really matter if the modulation is 2-ary, 4-ary, 8-ary, etc.

I don't know how big the codewords can get before the Map block chokes. At some point you'd do better to write a custom nearest-neighbor despreading block.

In [24]:
# Describe your code in this block
# code words packed into arbitrary-width integers
code = [ 0x22, 0xdd ]   # 8-bit code words (4 symbols of QPSK, perhaps)
codeword_length = 8     # number of bits (not symbols) in the code words
tiebreaker_mask = 0x01  # bit(s) to use if received bits are midway between the code words

# another example
# code = [ 0x12345, 0x0abcd ]
# codeword_length = 17
# tiebreaker_mask = 0x0008

In [25]:
assert(len(code) == 2)  # we decode only 0 or 1, so need only two codewords
codeword_range = 2**codeword_length

In [26]:
despread_map = [0]*codeword_range

In [27]:
def hamming_distance(a,b):
    ones = a ^ b
    return bin(ones).count('1')

In [28]:
for codeword in range(0,codeword_range):
    if hamming_distance(codeword, code[1]) > hamming_distance(codeword, code[0]):
        despread_map[codeword] = 1
    elif hamming_distance(codeword, code[1]) == hamming_distance(codeword, code[0]):
        if codeword & tiebreaker_mask:
            despread_map[codeword] = 1

In [29]:
print(despread_map)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
